---
### Image Gen

In [ ]:
!pip install diffusers transformers accelerate --upgrade

### SD Turbo: Text to Image

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
pipe.to("mps")

prompt = "an abstract figurative scupture depicting humanity, black marble"

for i in range(3):
    image = pipe(prompt=prompt, num_inference_steps=3, guidance_scale=0.5).images[0]
    image=image.resize((1024,1024))
    display(image)


### SD Turbo: Image to Image

In [ ]:
from diffusers import AutoPipelineForImage2Image, DiffusionPipeline
from diffusers.utils import load_image
import torch

pipe = AutoPipelineForImage2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
pipe.to("mps")

# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("mps")

refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("mps")

In [ ]:
from PIL import Image

init_image = load_image("https://rack01powering2.expertagent.co.uk/(S(w5vn2dffgx4lnlrgpuu1tv4r))/agencies/%7Bcb3e4606-1032-41d6-a3ce-b23a61fcc618%7D/%7B4f5bc5d4-5d0f-4679-8304-f0d12e7813ee%7D/main/IMG_99649AmyStreet.jpg").resize((512, 512))



display(init_image)

prompt = "scandinavian elegant interior design, white walls, wooden floors, large windows, plants, minimalistic"


In [ ]:
for i in range(1):
    # image = pipe(prompt, image=init_image, num_inference_steps=3, strength=.5, guidance_scale=0.5, denoising_end=.8, output_type="latent",).images[0]
    
    image = pipe(prompt, image=init_image, num_inference_steps=2, strength=.5, guidance_scale=.5, denoising_end=.8).images[0]
    image=image.resize((1024, 1024))

    
    display(image)

In [ ]:
refined = refiner(
    prompt=prompt,
    num_inference_steps=6,
    denoising_start=.85,
    image=image).images[0]
display(refined)

### Iterate Through Settings

In [ ]:
# Define lists for the parameters to iterate through
strength_values = [0.9, 0.7, 0.5]
num_inference_steps_values = [1, 2, 3]
guidance_scale_values = [0, 0.5, 1]

# Iterate through the parameters and generate comparison images
for strength_value in strength_values:
    for num_steps in num_inference_steps_values:
        for scale_value in guidance_scale_values:
            # Generate the image using the constant prompt and varying parameters
            if (num_steps*strength_value) >= 1:
                image = pipe(
                    prompt=prompt,
                    image=init_image,
                    num_inference_steps=num_steps,
                    strength=strength_value,
                    guidance_scale=scale_value,
                    denoising_end=0.8
                ).images[0]
            
                # Resize the image if needed
                # image = image.resize((1024, 1024))
                
                # Display or save the generated image here
                print (f"num_steps: {num_steps}, strength_value: {strength_value}, scale_value: {scale_value}")
                display(image)

### X1 SD

In [ ]:
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe.to("mps")

# if using torch < 2.0
# pipe.enable_xformers_memory_efficient_attention()

prompt = "add elements to the garden, a bench, a pond, grass"

images = pipe(prompt=prompt, 
              image=init_image, 
            #   negative_prompt="dog", 
              num_inference_steps=5, 
              guidance_scale=3, 
              strength=1).images[0]

display(images)


### + Refiner

In [ ]:
from diffusers import DiffusionPipeline
import torch

# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("mps")


refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("mps")

# Define how many steps and what % of steps to be run on each experts (80/20) here
n_steps = 20
high_noise_frac = 0.8

# prompt = "A majestic lion jumping from a big stone at night"

# run both experts
image = base(
    prompt=prompt,
    image=image,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images
image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,
    image=image,
).images[0]

display(image)

### SD Cascade

In [ ]:
!pip install git+https://github.com/kashif/diffusers.git
# @wuerstchen-v3


In [ ]:
import torch
from diffusers import StableCascadeDecoderPipeline, StableCascadePriorPipeline

device = "cpu"
num_images_per_prompt = 1

prior = StableCascadePriorPipeline.from_pretrained("stabilityai/stable-cascade-prior", torch_dtype=torch.bfloat16).to(device)
decoder = StableCascadeDecoderPipeline.from_pretrained("stabilityai/stable-cascade",  torch_dtype=torch.float16).to(device)

prompt = "Anthropomorphic cat dressed as a pilot"
negative_prompt = ""

prior_output = prior(
    prompt=prompt,
    height=512,
    width=512,
    negative_prompt=negative_prompt,
    guidance_scale=1.0,
    num_images_per_prompt=num_images_per_prompt,
    num_inference_steps=10
)
decoder_output = decoder(
    image_embeddings=prior_output.image_embeddings.half(),
    prompt=prompt,
    negative_prompt=negative_prompt,
    guidance_scale=0.0,
    output_type="pil",
    num_inference_steps=5
).images

for image in decoder_output:
    display(image)

#Now decoder_output is a list with your PIL images


In [ ]:
!pip install transformers accelerate -q
!pip install git+https://github.com/huggingface/diffusers@@shap-ee



In [ ]:
import torch
from diffusers import ShapEPipeline
from diffusers.utils import export_to_gif


ckpt_id = "openai/shap-e"
pipe = ShapEPipeline.from_pretrained(repo).to("cuda")


guidance_scale = 15.0
prompt = "a shark"
images = pipe(
    prompt,
    guidance_scale=guidance_scale,
    num_inference_steps=64,
    size=256,
).images

gif_path = export_to_gif(images, "shark_3d.gif")


---
---
### Other

---
---
### Music Gen

In [ ]:
from transformers import pipeline
import scipy

synthesiser = pipeline("text-to-audio", "facebook/musicgen-small")

music = synthesiser("lo-fi music with a soothing melody", forward_params={"do_sample": True}, max_new_tokens =50)

scipy.io.wavfile.write("musicgen_out.wav", rate=music["sampling_rate"], data=music["audio"])


### More Control

In [ ]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration
import torch

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small").to("cpu")

inputs = processor(
    text=["soothing ambient insturmental meditation music"],
    padding=True,
    return_tensors="pt", 
)

audio_values = model.generate(**inputs, max_new_tokens=300)


### Play Here

In [ ]:
from IPython.display import Audio

sampling_rate = model.config.audio_encoder.sampling_rate
Audio(audio_values[0].numpy(), rate=sampling_rate)


### Save Audio

In [ ]:
import scipy

sampling_rate = model.config.audio_encoder.sampling_rate
scipy.io.wavfile.write("musicgen_out.wav", rate=sampling_rate, data=audio_values[0, 0].numpy())


### Distill

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download

base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
ckpt = "sdxl_lightning_4step_lora.safetensors" # Use the correct ckpt for your step setting!

# Load model.
pipe = StableDiffusionXLPipeline.from_pretrained(base, torch_dtype=torch.float16, variant="fp16").to("cpu")
pipe.load_lora_weights(hf_hub_download(repo, ckpt))
pipe.fuse_lora()

# Ensure sampler uses "trailing" timesteps.
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")

# Ensure using the same inference steps as the loaded model and CFG set to 0.
pipe("A girl smiling", num_inference_steps=4, guidance_scale=0).images[0].save("output.png")


In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
ckpt = "sdxl_lightning_4step_unet.safetensors" # Use the correct ckpt for your step setting!

# Load model.
unet = UNet2DConditionModel.from_config(base, subfolder="unet").to("cpu", torch.float16)
unet.load_state_dict(load_file(hf_hub_download(repo, ckpt), device="cpu"))
pipe = StableDiffusionXLPipeline.from_pretrained(base, unet=unet, torch_dtype=torch.float16, variant="fp16").to("cuda")

# Ensure sampler uses "trailing" timesteps.
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")

# Ensure using the same inference steps as the loaded model and CFG set to 0.
pipe("A girl smiling", num_inference_steps=4, guidance_scale=0).images[0].save("output.png")


In [3]:
from TTS.api import TTS

ModuleNotFoundError: No module named 'TTS'

In [2]:
from TTS.api import TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)

# generate speech by cloning a voice using default settings
tts.tts_to_file(text="It took me quite a long time to develop a voice, and now that I have it I'm not going to be silent.",
                file_path="output.wav",
                speaker_wav="/output/speaker.wav",
                language="en")


ModuleNotFoundError: No module named 'TTS'

---
---
### SENTIMENTS: